# Excercse CIFAR10

1. complete the training process
2. Get the MAX accuracy and MIN loss of your final model

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, models, layers, utils, activations, losses, optimizers, metrics
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
tf.__version__

In [ ]:
# Define Parameters
num_classes = 10
img_size = 32

#### Prepare Data

In [ ]:
# download from google drive
!gdown --id '1RaH2V6FdhBxjUbREJMY2SdLS-O_Qhkq7' --output cifar10.npz

In [ ]:
# Load Data from file
with np.load('cifar10.npz', allow_pickle=True) as f:
    x_train, y_train = f['x_train'], f['y_train']
    x_test = f['x_test']

# Data shape
x_train.shape, y_train.shape, x_test.shape

In [ ]:
# split dataset
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)

x_train.shape, x_val.shape, x_train.shape, x_val.shape

In [ ]:
# Flatten img to a vector
x_train = x_train.reshape(x_train.shape[0], -1)
x_val = x_val.reshape(x_val.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)

# normalization to 0 ~ 1
x_train = x_train / 255.
x_val = x_val / 255.
x_test = x_test / 255.

# one hot encoding
y_train = utils.to_categorical(y_train, num_classes=num_classes)
y_val = utils.to_categorical(y_val, num_classes=num_classes)

x_train.shape, x_val.shape, y_train.shape, y_val.shape

#### Build Model

In [ ]:
# Build your own model
model = models.Sequential([layers.InputLayer((img_size*img_size*3)),
                           layers.Dense(32, activation='relu'),
                           layers.Dense(32, activation='relu'),
                           layers.Dense(64, activation='relu'),
                           layers.Dense(64, activation='relu'),
                           layers.Dense(10, activation='softmax'),
])

In [ ]:
model.summary()

#### Training

In [ ]:
# Setting for training
model.compile(loss=losses.categorical_crossentropy, # loss function
              optimizer=optimizers.SGD(),
              metrics=['accuracy']
              )

In [ ]:
logs = model.fit(x_train, y_train,
                batch_size=128,
                epochs=10,
                validation_data=(x_val, y_val))

In [ ]:
prediction = model.predict(x_test[:1])
print('model prediction: ', prediction)
print('max probabolity index: ', np.argmax(prediction, axis=-1))

In [ ]:
history = logs.history
print(history)

In [ ]:
plt.plot(history['accuracy'])
plt.plot(history['val_accuracy'])
plt.legend(['accuracy', 'val_accuracy'])
plt.title('accuracy')

#### Get max accuracy from history['val_accuracy']

In [ ]:
max_acc = np.max(history['val_accuracy'])
print(max_acc)

In [ ]:
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.title('loss')

#### Get min loss from history['val_loss']

In [ ]:
min_loss = np.min(history['val_loss'])
print(min_loss)

#### Metrics

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
y_true = np.argmax(y_val, axis=-1) # Ground Truth
y_pred = np.argmax(model.predict(x_val), axis=-1) # Model prediction
print(y_true.shape, y_pred.shape)


In [ ]:
# Classification Report
# Ref: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html
label_names = [str(i) for i in range(10)] # ['0' ~  '9']
print(classification_report(y_true, y_pred, target_names=label_names))

In [ ]:
# Confusion Matrix: 
# Row: Truth
# Column: Prediction
# Ref: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
print(confusion_matrix(y_true, y_pred))

#### Generate file for Kaggle

In [ ]:
df = pd.DataFrame()
df['Id'] = [str(i) for i in range(len(x_test))]
predictions = model.predict(x_test)
predictions = np.argmax(predictions, axis=-1)
df['Category'] = predictions
df.to_csv('result.csv', index=None)
df